In [1]:
# general imports

import io
import sys
import subprocess 
import copy

import numpy as np
import pyvista as pv

import galois
import json

# make sure the user has at least Python 3.7, which we need for some capture_output
assert sys.version_info.major == 3, "This requires at least Python 3.7"
assert sys.version_info.minor >= 7, "This requires at least Python 3.7"



In [2]:
# some classes

class Position:
    def __init__self():
        self.x = None
        self.y = None
        self.z = None

class Value:
    def __init__self():
        self.v0 = None
        self.v1 = None
        self.v2 = None    
        
class Node:
    def __init__self():
        self.value = None
        self.edges = None
        self.lexpos = None
        self.goodpos = None
        self.cakepos = None


In [3]:
# finite field arithmetic tables and 3-D dot product calculation

def make_GFq_arith_table(q):
# addition and multiplication tables in GF(q)
    GFq_add = []
    GFq_mul = []
    GF = galois.GF(q)
    for i in range(q):
        x_vec = []
        y_vec = []
        for j in range(q):
            x_vec.append(i)
            y_vec.append(j)
        x = GF(x_vec)
        y = GF(y_vec)
        GFq_add.append(json.loads(str(x+y)))
        GFq_mul.append(json.loads(str(x*y)))
    return GFq_add, GFq_mul
    
def dot_product_3d(v,w,q):
# v and w are orthogonal in GF(q)^3 if their dot product is 0 in GF(q) arithmetic
    s=[]
    for i in range(3):
        s.append(GFq_mul[v[i]][w[i]])
    d = GFq_add[GFq_add[s[0]][s[1]]][s[2]]
    return d


In [4]:
# graph: generate vertices and incidence matrix
# these return pointers to the elements in the subsets
# the pointer is the lexicographic position of the left-normalized element of (F_q)^3

def generate_vertices(q):
# this generates the q^2+q+1 vertices of ER_q
# vertex_value are the left-normalized vectors of ER_q, in lexicographic order
    vertex_value = []
    for i in range(int(q**3)):
    # decomposes an integer into its GF(q)^3 vector
        new_point = []
        new_point.insert(0, int(i%q))
        new_point.insert(0, int(((i-new_point[0])/q)%q))
        new_point.insert(0, int(((i-new_point[0]*q-new_point[1])/q**2)%q))
    # append those that are left-normalized
        if (new_point[0]==1):
            vertex_value.append(new_point)
        elif (new_point[0]==0 and new_point[1]==1):
            vertex_value.append(new_point)
        elif (new_point[0]==0 and new_point[1]==0 and new_point[2]==1):
            vertex_value.append(new_point)            
    return vertex_value

def incidence_matrix(vertices_values):
# incidence_matrix[i,j] is 1 if i is orthogonal to j and 0 otherwise
    inc_matrix = []
    num_vertices = len(vertices_values)
    for i in range(num_vertices):
        row = [0]*num_vertices
        inc_matrix.append(row)
    for i in range(num_vertices):
        for j in range(i,num_vertices):
            d = dot_product_3d(vertices_values[i], vertices_values[j],q)
            if d==0:
                inc_matrix[i][j]=1
                inc_matrix[j][i]=1
    return inc_matrix


In [5]:
# graph: generate  important vertex subsets:
# L (the initial quadric), W (all quadrics except the starter one), C, V1 and V2
# also the center elements (given L)
# these lists are pointers to the elements in the subsets
# the pointer is the lexicographic position of the left-normalized element of (F_q)^3

def generate_W(inc_matrix, starter_quad_index):
    W=[]
    L=[]
    num_vertices = len(inc_matrix[0])
    for i in range(num_vertices):
        if inc_matrix[i][i]==1:
            W.append(i)
    L.append(W[starter_quad_index])
    W.pop(starter_quad_index)
    return L, W

def generate_V1(L, W, inc_matrix):
    C=[]
    V1=[]
    num_vertices = len(inc_matrix[0])
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and inc_matrix[i][L[0]]==1:
            C.append(i)     
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and (i not in C):
            for j in W:
                if inc_matrix[i][j]==1:
                    V1.append(i)
                    break
    return (C,V1)

def generate_V2(L, W, C, V1, num_vertices):
    V2=[]
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and (i not in C) and (i not in V1):
            V2.append(i)
    return V2

def generate_center(inc_matrix, l):
# gives the cluster vertices defined by a quadric
    # choose a center element, must be perp to l but not self-perp, i.e., not in W
    centerpointers = []
    for i in range(len(inc_matrix[0])):
        if inc_matrix[l][i]==1 and inc_matrix[i][i]==0:
            centerpointers.append(i)
    return centerpointers



In [6]:
# graph: retrieve parts of the graph that are of interest

def get_positions_from_pointers(vertex_position, pointers):
# this grabs a bunch of positions corresponding to the pointers
    out_positions = []
    for i in range(len(pointers)):
        out_positions.append(vertex_position[pointers[i]])
    return out_positions

def get_edges(inc_matrix, pointer_set):
# get all of the edges out of a given pointer_set
    clusteredges = []
    for i in range(len(pointer_set)):
        for j in range(i+1,len(pointer_set)):
            if inc_matrix[pointer_set[i]][pointer_set[j]]==1:
                clusteredges.append([2,pointer_set[i],pointer_set[j]])
    return clusteredges

def get_edges_between_ptrsets(inc_matrix, pointer_set_0, pointer_set_1):
# get all of the edges between two pointer_sets
    clusteredges = []
    for i in range(len(pointer_set_0)):
        for j in range(len(pointer_set_1)):
            if inc_matrix[pointer_set_0[i]][pointer_set_1[j]]==1:
                clusteredges.append([2,pointer_set_0[i],pointer_set_1[j]])
    return clusteredges

def get_cluster(inc_matrix, vpos, l, c):
# get the cluster generated from a given quadric and center element
    clusterpointers=[c]
    for i in range(len(inc_matrix[0])):
        if inc_matrix[c][i]==1 and inc_matrix[i][i]==0:
            clusterpointers.append(i)
    clustervpos = get_positions_from_pointers(vpos, clusterpointers)
    clusteredges = get_edges(inc_matrix, clusterpointers) 
    return (clusterpointers, clustervpos, clusteredges)

def get_all_clusters(inc_matrix, l):
# get the cluster generated from a given quadric and center element
    centerptrs = generate_center(inc_matrix, l)
    clusters=[]
    for i in range(len(centerptrs)):
        clusterptrs=[centerptrs[i]]
        for j in range(len(inc_matrix[0])):
            if inc_matrix[centerptrs[i]][j]==1 and inc_matrix[j][j]==0:
                clusterptrs.append(j)
        clusters.append(clusterptrs)
    return clusters

def append_edges(edges, opacity, color, line_width):
# append a set of edges to the current edge_sets list, along with their positions and drawing details
    edge_sets.append(edges)    
    edge_opacities.append(opacity)
    edge_colors.append(color)
    line_widths.append(line_width)


In [7]:
# viz stuff
# Let's define some colors
RED = '#FF0000'
YELLOW = '#CCCC00'
GREEN = '#00DD00'
CYAN = '#00FFFF'
BLUE = '#0000FF'
MAGENTA = '#CC00CC'

LT_RED = '#FF3333'
DK_RED = '#770000'
ORANGE = '#FF8400'
LT_GREEN = '#44FF44'
DK_GREEN = '#009900'
NEON_GREEN = '#00FE94'
LT_BLUE = '#0088FF'
NEON_BLUE = '#7DF9FF'
LIGHT_COLOR = 'white'
BACKGROUND_COLOR = 'white'
GREY = '#AAAAAA'
LT_GREY = '#CCCCCC'

L_color = RED
W_color = DK_RED
C_color = LT_GREEN
V1_color = DK_GREEN
V2_color = LT_BLUE
# these next are for the lex display if desired
# vertex_size = 70
# W_color = RED
# C_color = GREEN
# V1_color = GREEN


def visualize_graph(q, vertex_value, L, W, C, V1, V2, vpos, 
                    edge_sets, edge_opacities, edge_colors, line_widths, 
                    fname, camera_pos, camera_rot , show_labels, caption):
    print(caption)
    if q<17:
        edge_opacities[0] = 0.08
    elif q<27:
        edge_opacities[0] = 0.04
    elif q<47:
        edge_opacities[0] = 0.01
    elif q<63:
        edge_opacities[0] = 0.005
    elif q<129:
        edge_opacities[0] = 0.0005
    else:
        edge_opacities[0] = 0.0001
    vertex_size = 25
    vertex_size_smaller = vertex_size

    # camera_pos views: xy, xz, yz, yx, zx, zy, iso, can also define a camera_pos = (x,y,z) tuple
#     camera_pos = 'zx'
#     camera_rot = [0,5,15]
#     # overhead view
#     camera_pos = 'xy'
#     camera_rot = [0,0,180]
    pl = pv.Plotter(lighting=None, window_size=(2000, 2000))
    light1 = pv.Light(position=(10, 10., -5.0),
                       focal_point=(0, 0, 0),
                       color=LIGHT_COLOR,  # Color temp. 5400 K
                       intensity=1.5)
    light2 = pv.Light(position=(-10, 10.0, -5.0),
                       focal_point=(0, 0, 0),
                       color=LIGHT_COLOR,  # Color temp. 2850 K
                       intensity=0.75)
#     pl.add_light(light1)  # Lighting effect to be cast onto the object
#     pl.add_light(light2)  # Second Lighting effect to be cast onto the object
    hlight = pv.Light(light_type='headlight')
    pl.add_light(hlight)  # Second Lighting effect to be cast onto the object
    pl.set_background(BACKGROUND_COLOR)  # Set the background 
            
    # pdata is the basic vertex data positions
    pdata = pv.PolyData(vpos)     
# this adds point labels of vector values -- but placed on top of the points
    if (show_labels):
        point_labels = []
        for i in range(len(vertex_value)):
            point_labels.append(vertex_value[i])        
        pl.add_point_labels(pdata, point_labels, italic=False, bold=True, font_size=30,
                            point_color='black', point_size=0, text_color='black', 
                            shape=None, 
                            render_points_as_spheres=True,
                            always_visible=True, shadow=False)  

# this adds the other edge sets    
    other_pdata = []
    for i in range(len(edge_sets)):
        other_pdata.append(pv.PolyData(vpos))
        other_pdata[i].lines = edge_sets[i]
        pl.add_mesh(other_pdata[i],
                    color = edge_colors[i],
                    opacity = edge_opacities[i],
                    point_size=0,
                    line_width = line_widths[i],
                    render_points_as_spheres=True
        )
        
    L_vertex_position=[]
    for i in L:
        L_vertex_position.append(vpos[L[0]])
    L_cloud = pv.PolyData(L_vertex_position)
    pl.add_mesh(L_cloud, 
                color=L_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    W_vertex_position=[]
    for i in W:
        W_vertex_position.append(vpos[i])
    W_cloud = pv.PolyData(W_vertex_position)
    pl.add_mesh(W_cloud, 
                color=W_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    C_vertex_position=[]
    for i in C:
        C_vertex_position.append(vpos[i])
    C_cloud = pv.PolyData(C_vertex_position)
    pl.add_mesh(C_cloud, 
                color=C_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    V1_vertex_position=[]
    for i in V1:
        V1_vertex_position.append(vpos[i])
    V1_cloud = pv.PolyData(V1_vertex_position)
    pl.add_mesh(V1_cloud, 
                color=V1_color, 
                point_size=vertex_size_smaller,
                render_points_as_spheres=True
               )
    V2_vertex_position=[]
    for i in V2:
        V2_vertex_position.append(vpos[i])
    V2_cloud = pv.PolyData(V2_vertex_position)
    pl.add_mesh(V2_cloud, 
                color=V2_color, 
                point_size=vertex_size_smaller,
                render_points_as_spheres=True
               )
    
    # Views: xy, xz, yz, yx, zx, zy, iso 
    pl.camera_position = camera_pos
    pl.camera.roll = camera_rot[0]
    pl.camera.azimuth = camera_rot[1]
    pl.camera.elevation = camera_rot[2]
    pl.camera.zoom(1)   
    # Save the plot as a screenshot.  We can change the output resolution if desired
    pl.screenshot(fname, window_size=[2000,2000])
    #pl.add_point_labels(vpos_sets[0], range(len(vpos_sets[0])), font_size=100)
    #pl.show_axes()
    pl.show()


In [8]:
# graph: layout 
# positioning of vertices, the edges automatically follow

def init_pos(num_vertices):
# vertex_position gives the initial positions of q^2+q+1 points equidistant on the unit circle
# you can change vertex_position to place these elsewhere in the space, for example into 3-space
    vpos = []
    # vertices locations
    for i in range(num_vertices):
        angle_increment = 2*np.pi/num_vertices    
        vpos.append([np.sin(i*angle_increment), np.cos(i*angle_increment), 0])
    return vpos

def reset_pos(vertex_position):
    vpos_temp = copy.deepcopy(vertex_position)
    return vpos_temp

def graph_to_clusters_good(q, vertex_position, inc_matrix, L, W, C, V1, V2, clusters):
# this permutes the positions of the vertices in a way that helps to show how clusters are made
    num_vertices = len(vertex_position)
    vpos_temp = copy.deepcopy(vertex_position)
    # place the starting quad (L[0]) (q+1)/2 to the left of the top
    vpos_temp[L[0]] = vertex_position[num_vertices-int((q+1)/2)]
    # place the elements of C to spread over the top of the circle
    end_C = int((q-1)/2)
    for i in range(len(C)):
        vpos_temp[C[i]] = vertex_position[(i-end_C)%num_vertices]
    current_pos = end_C+1
    # move the cluster elements of V1 to points corresponding to their cluster centers from C
    # want them to be more or less directly below their center
    for i in range(len(C)):        
#         V1_Ci = list(set(V1) & set(clusters[len(C)-1-i]))
#         V2_Ci = list(set(V2) & set(clusters[len(C)-1-i]))        
        V1_Ci=[]
        V2_Ci=[]
        # collect the elements of V1 and V2 that are in C_i
        for j in range(len(V1)):
            if inc_matrix[V1[j]][C[len(C)-1-i]]==1:
                V1_Ci.append(V1[j])
            if inc_matrix[V2[j]][C[len(C)-1-i]]==1:
                V2_Ci.append(V2[j])
        # make triangles
        current_pos+=1
        if (q%4 != 1):
            for j in range(int((q-1)/2)):
                # take the jth V1_temp and match it up with the V2 that is orthogonal
                # here's the jth V1
                vpos_temp[V1_Ci[j]] = vertex_position[current_pos]
                current_pos += 1
                # take the orthogonal V2
                for k in range(len(V2_Ci)):
                    if inc_matrix[V1_Ci[j]][V2_Ci[k]]==1:
                        vpos_temp[V2_Ci[k]] = vertex_position[current_pos]
                        current_pos += 1
                        break
        else: # q%4 == 1
            # will have (q-1)/4 pairs of V1 only tris, and (q-1)/4 pairs of V2 only tris
            for j in range(int((q-1)/4)):
                # all the tris are within V1 or within V2
                # pick the first V1 and V2 (any one not picked will do)
                vpos_temp[V1_Ci[0]] = vertex_position[current_pos]
                current_pos += 1
                vpos_temp[V2_Ci[0]] = vertex_position[current_pos]
                current_pos += 1
                #pick twins of V1_temp[0]
                for k in range(1,len(V1_Ci)):
                    if inc_matrix[V1_Ci[0]][V1_Ci[k]]==1:
                        vpos_temp[V1_Ci[k]] = vertex_position[current_pos]
                        current_pos += 1
                        V1_Ci.pop(k)
                        V1_Ci.pop(0)
                        break
                #pick twins of V2_temp[0]
                for k in range(1,len(V2_Ci)):
                    if inc_matrix[V2_Ci[0]][V2_Ci[k]]==1:
                        vpos_temp[V2_Ci[k]] = vertex_position[current_pos]
                        current_pos += 1
                        V2_Ci.pop(k)
                        V2_Ci.pop(0)
                        break                    
    # these will be interspersed between each "rack" C_i of V1 and V2 points
    C_temp=[]
    for i in range(len(C)):
        C_temp.append(C[i])
    mid_C = int((len(C)-1)/2)
    C_temp.pop(mid_C)
    for i in range(len(W)):
        if inc_matrix[W[i]][C[mid_C]]==1:
            vpos_temp[W[i]] = vertex_position[(end_C+1)%num_vertices]
    for j in range(len(C_temp)):
        W_temp=[]
        for i in range(len(W)):
            if inc_matrix[W[i]][C_temp[len(C_temp)-1-j]]==1:
                W_temp.append(W[i])                                
        for i in range(len(W_temp)):
            vpos_temp[W_temp[i]] = vertex_position[(end_C+1+(j+1)*q)%num_vertices]           
    return vpos_temp

def graph_to_clusters_pos_2(vertex_position, L):
# move the arbitrary quadric element in L outside and above the circle
    vpos_temp = copy.deepcopy(vertex_position)
    height_L = 1.25
    vpos_temp[L[0]] = [0.0,height_L,0.0]  
    return vpos_temp

def graph_to_clusters_pos_3(vertex_position, W):
# move the arbitrary quadric element in L outside and above the circle
    vpos_temp = copy.deepcopy(vertex_position)
    vpos_temp[W[1]][0] = vertex_position
    height_W = 1.25
    dist_W = 0.5
    for i in range(len(W)):
        vpos_temp[W[i]][0] = dist_W - i*dist_W/q
        vpos_temp[W[i]][1] = height_W
    return vpos_temp

def place_in_circle(ptrlist, vertex_position, starting_angle, radius, height):
# places the elements of ptrlist in a circle of radius and height
    vpos_temp = copy.deepcopy(vertex_position)
    num_ptrlist = len(ptrlist)
    angle = -2*np.pi/(num_ptrlist)
    for i in range(len(ptrlist)):
        vpos_temp[ptrlist[i]][0]= radius*np.sin(i*angle+starting_angle)
        vpos_temp[ptrlist[i]][1]= radius*np.cos(i*angle+starting_angle)
        vpos_temp[ptrlist[i]][2]= height
    return vpos_temp

def permute_pos(vpos, K, permutation):
# takes a permutation of the form [[x00, x01, ... x0n], ... , [xs0, xs1, ... xsm]]
# s+1 self-contained permutations, moving however many elements
# permutes the positions of K as per the permutation
    num_K = len(K)
    num_perms = len(permutation)
    #save original positions of K out
    vpos_temp_list = []
    for i in range(num_K):
        vpos_temp = copy.deepcopy(vpos[K[i]])
        vpos_temp_list.append(vpos_temp)
    for k in range(num_perms):
        num_perm = len(permutation[k])
        # move src element position to dest element position, for each move in the permutation
        for i in range(num_perm):
            src = i
            dest = (i+1)%num_perm
            for j in range(3):
                vpos[K[permutation[k][dest]]][j] = vpos_temp_list[permutation[k][src]][j]    
    
def graph_layercake_pos(vertex_position, L, W, C, V1, V2):
    vpos_temp = copy.deepcopy(vertex_position)
    starting_angle = 0    
# Move the W away from the viewer and into the center
    radius_W= .25
    height_W = -1.0
    vpos_temp = place_in_circle(L+W, vpos_temp, starting_angle, radius_W, height_W)
# This permutation on the quadrics is used in the paper to cleanly show quadrics connections for q=7
    if len(W) == 7:
        permute_pos(vpos_temp, L+W, [[0,1,2,4,6,7]])
# Move the C elements into the center
    radius_C= 1.5*radius_W
    angle_C = 2*np.pi/(len(C))
    starting_angle_C = starting_angle-0.5*angle_C
    height_C = 0        
    vpos_temp = place_in_circle(C, vpos_temp, starting_angle_C, radius_C, height_C)
# Move the V2 elements in z toward the viewer at positive z
    for i in range(len(V2)):
        vpos_temp[V2[i]][2] += 1.0 
    return vpos_temp


In [9]:
#####    these are the prime powers q to be run         ##### 
#####    this algorithm is for ODD PRIME POWERS only    ##### 

# Some smaller primes
#prime_power_list = [3,5,7,9,11,13,17,19,23,25,27,29,31,37,41,43,47,49,53,59,61]

# Primes of interest.
# Graphs for these primes support [556,993,2257,3783,16257] routers, having radixes [24,32,48,62,128]. 
# May want to change the opacity on edges and maybe the line width in the viz.
#prime_power_list = [23,31,47,61,127]  

#prime_power_list = [3,7]  # used for the paper viz

prime_power_list = [7,9,11]


for q in prime_power_list:
    
    print('GF(' + str(q) + ')')
    # where to save screenshots
    directory = "./ER_graphs/"
    fn_prefix = directory + "ER" + str(q)

    # the GF(q) addition and multiplication tables
    GFq_add,GFq_mul = make_GFq_arith_table(q)    
    num_vertices = q**2+q+1

    # the left-normalized 3-vectors which are the vertices, in lexicographic order 
    vertex_value = generate_vertices(q)
    # the incidence matrix, calculated using the dot product over GF(q)
    inc_matrix = incidence_matrix(vertex_value)
    # pointers for the vertices
    vpointers = [i for i in range(len(vertex_value))]
        
    # all the edge sets, opacities, colors, line widths that will be visualized
    edge_sets = []
    edge_opacities = []
    edge_colors = []
    line_widths = []
    # full set of edges
    edges = get_edges(inc_matrix, vpointers)
    append_edges(edges, 1, 'black', 3)
    
    # generate the important vector subsets
    # L is the arbitary quadric that starts things off
    # W is the rest of the quadrics
    quad_starter_element = 0   # which quadric element will we choose to start things off?
    L,W = generate_W(inc_matrix, quad_starter_element)
    # C is the set of centers: the vectors that are adjacent to the element in L
    # V1 is the set of all other vectors adjacent to some quadric
    C,V1 = generate_V1(L, W, inc_matrix)
    # V2 is the rest of the vectors, not adjacent to any quadric
    V2 = generate_V2(L, W, C, V1, num_vertices)
    clusters = get_all_clusters(inc_matrix, L[0])
    
# Vertex positions. These lay out the graph. These are some good ones.
    # the lexicographic vertex positions, clockwise around the circle
    vertex_position_lex = init_pos(num_vertices)
    # the basic "good" layout with the edges, can be easily teased apart to show cluster construction
    vertex_position_good = graph_to_clusters_good(q, vertex_position_lex, inc_matrix, L, W, C, V1, V2, clusters)
    # the layer cake
    vertex_position_cake = graph_layercake_pos(vertex_position_good, L, W, C, V1, V2)
    
    # Visualize various layouts

# Viz: the initial lexicographic clockwise layout
    visualize_graph(q, vertex_value, L, W, C, V1, V2, 
                    vertex_position_lex, edge_sets, edge_opacities, edge_colors, line_widths, 
                    fn_prefix + "_lex_cw", 'xy', [0,0,0], False, 
                    'GF('+str(q)+'): Lexicographic layout')

    # Viz: the first graph_to_clusters layout: nicer looking than lexicographic clockwise layout
    visualize_graph(q, vertex_value, L, W, C, V1, V2, 
                    vertex_position_good, edge_sets, edge_opacities, edge_colors, line_widths, 
                    fn_prefix + "_g2c_1", 'xy', [0,0,0], False, 
                    'GF('+str(q)+'): Structured layout')
# Viz: the layer cake
#     visualize_graph(q, vertex_value, L, W, C, V1, V2, 
#                      vertex_position_cake, edge_sets, edge_opacities, edge_colors, line_widths, 
#                      fn_prefix + "_cake_1cluster_3D", 'zx', [0,0,20], False, 
#                     'GF('+str(q)+'): Layer cake, all edges')

    clusterpointers = []   
# Edges: main cluster, heavy black lines
# sets clusterpointers[0]
    cluster_index = 0  # which center element will we choose for the center?
    clustervptrs, clustervpos, clusteredges = get_cluster(inc_matrix, vertex_position_good, L[0], C[cluster_index])
#     clusterpointers.append(clustervptrs)
    clusterpointers.append(clusters[0])
    append_edges(clusteredges, 1.0, 'black', 3)    
# Edges: quadric edge in RED
    quad_edge = [[2, L[0], C[cluster_index]]]
    append_edges(quad_edge, 1.0, RED, 4)
       
# Viz: the layer cake with all cluster edges plus the quadric 
#     visualize_graph(q, vertex_value, L, W, C, V1, V2, 
#                     vertex_position_cake, edge_sets, edge_opacities, edge_colors, line_widths, 
#                     fn_prefix + "_cake_1cluster_3D", 'zx', [0,0,20], False, 
#                     'GF('+str(q)+'): More structured layout, transparent edges plus dark cluster edges')

# Edges: other clusters
# sets clusterpointers[1]
    # show only one other
    num_clusters = 1
    # show them all
#     num_clusters = q
    for i in range(1,num_clusters+1):
        clustervptrs, clustervpos, clusteredges = get_cluster(inc_matrix, vertex_position_good, L[0], C[num_clusters-(i+1)])
#         clusterpointers.append(clusters[i])
        clusterpointers.append(clustervptrs)
        append_edges(clusteredges, 1.0, 'black', 3)

# Viz: other clusters
#     visualize_graph(q, vertex_value, L, W, C, V1, V2, 
#                     vertex_position_cake, edge_sets, edge_opacities, edge_colors, line_widths, 
#                     fn_prefix + "_cake_allclusters_3D", 'zx', [0,0,20], False, 
#                     'GF('+str(q)+'): Layer cake, transparent edges plus two clusters with dark cluster edges')
# Viz: other clusters in overhead view
#     visualize_graph(q, vertex_value, L, W, C, V1, V2, 
#                     vertex_position_cake, edge_sets, edge_opacities, edge_colors, line_widths, 
#                     fn_prefix + "_cake_allclusters_3D", 'xy', [0,0,180], False, 
#                     'GF('+str(q)+'): Layer cake, overhead view, transparent edges plus some number of clusters with dark cluster edges')

# Edges: between cluster0 to the V1 elts of cluster1 in GREEN (there are no such edges for q=3)
    if (q != 3):
        clusterptrs_to_V1 = list(set(clusterpointers[1]) & set(V1))
        between_edges = get_edges_between_ptrsets(inc_matrix, clusterpointers[0], clusterptrs_to_V1)
        append_edges(between_edges, 1.0, GREEN, 4)   
# Edges: between cluster0 to the V2 elts of cluster1 in BLUE
    clusterptrs_to_V2 = list(set(clusterpointers[1]) & set(V2))
    between_edges = get_edges_between_ptrsets(inc_matrix, clusterpointers[0], clusterptrs_to_V2)
    append_edges(between_edges, 1.0, LT_BLUE, 4)    
# Edges: between cluster0 and W in RED
    Quadrics = W+L
    between_quad_ci_edges = get_edges_between_ptrsets(inc_matrix, clusterpointers[0], Quadrics)
    append_edges(between_quad_ci_edges, 1.0, RED, 4)       
# make the original edges completely transparent
    edge_opacities[0] = 0  

# Viz: all connections between the original cluster and its neighbor, and with the quadrics
    visualize_graph(q, vertex_value, L, W, C, V1, V2, 
                    vertex_position_cake, edge_sets, edge_opacities, edge_colors, line_widths, 
                    fn_prefix + "_cake_2clusters_edgesbetween_3D", 'zx', [0,0,20], False, 
                    'GF('+str(q)+'): Layer cake, no main edges, two clusters with dark cluster edges and colored edges joining')



GF(7)
GF(7): Lexicographic layout


FileNotFoundError: [Errno 2] No such file or directory: '/Users/lmonroe/work/projects/sphere-sampling/analysis/old_er/ER_graphs/ER7_lex_cw.png'